In [1]:
import numpy as np
import pandas as pd
from scipy.misc import *
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
import os

Using TensorFlow backend.


In [2]:
data_dir=os.path.abspath('D:\\Analytics Vidya\\Deep Learning\\Loan Prediction')

In [4]:
train=pd.read_csv(os.path.join(data_dir,'train.csv'))

In [5]:
test=pd.read_csv(os.path.join(data_dir,'test.csv'))

In [6]:
sub=pd.read_csv(os.path.join(data_dir,'sub.csv'))

In [7]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [10]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [11]:
train.shape

(614, 13)

In [13]:
train1=train
test1=test
sub1=sub

In [ ]:
train['Gender']=train['Gender'].replace({'Male':1,'Female':0})

In [37]:
train['Dependents']=train['Dependents'].replace({0:0,1:1,2:2,'3+':3,'NaN':0})

In [40]:
train['Gender'].fillna(train['Gender'].mode()[0], inplace=True) 
train['Married'].fillna(train['Married'].mode()[0], inplace=True)
train['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True) 
train['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True)
train['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)

In [41]:
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
train['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

In [42]:
test['Gender'].fillna(train['Gender'].mode()[0], inplace=True)
test['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True) 
test['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True) 
test['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)
test['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
test['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

In [43]:
train['LoanAmount_log'] = np.log(train['LoanAmount']) 
test['LoanAmount_log'] = np.log(test['LoanAmount'])

In [45]:
train=train.drop('Loan_ID',axis=1) 
test=test.drop('Loan_ID',axis=1)

In [46]:
X = train
y = train.Loan_Status

In [74]:
y=y.replace({'Y':1,'N':0})

In [48]:
X_test=test1

In [49]:
X_test.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'LoanAmount_log'],
      dtype='object')

In [52]:
X.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'LoanAmount_log'],
      dtype='object')

In [91]:
X['Married']=X['Married'].replace({'No':0,'Yes':1})
X['Education']=X['Education'].replace({'Graduate':1,'Not Graduate':0})
X['Property_Area']=X['Property_Area'].replace({'Urban':1,'Rural':0,'Semiurban':2})
X['Self_Employed']=X['Self_Employed'].replace({'No':0,'Yes':1})

In [95]:
X_test['Married']=X_test['Married'].replace({'No':0,'Yes':1})
X_test['Education']=X_test['Education'].replace({'Graduate':1,'Not Graduate':0})
X_test['Property_Area']=X_test['Property_Area'].replace({'Urban':1,'Rural':0,'Semiurban':2})
X_test['Self_Employed']=X_test['Self_Employed'].replace({'No':0,'Yes':1})

In [96]:
X_test['Gender']=X_test['Gender'].replace({'Male':1,'Female':0})

In [97]:
X_test.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Credit_History', 'Property_Area', 'LoanAmount_log', 'Total_Income',
       'Total_Income_log', 'EMI', 'Balance Income'],
      dtype='object')

In [98]:
X.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Credit_History', 'Property_Area', 'LoanAmount_log', 'Total_Income',
       'Total_Income_log', 'EMI', 'Balance Income'],
      dtype='object')

In [51]:
X=X.drop('Loan_Status',axis=1)

In [53]:
X['Total_Income']=X['ApplicantIncome']+X['CoapplicantIncome']
X_test['Total_Income']=X_test['ApplicantIncome']+X_test['CoapplicantIncome']
X['Total_Income_log'] = np.log(X['Total_Income'])  
X_test['Total_Income_log'] = np.log(X_test['Total_Income'])
X['EMI']=X['LoanAmount']/X['Loan_Amount_Term']
X_test['EMI']=X_test['LoanAmount']/X_test['Loan_Amount_Term']
X['Balance Income']=X['Total_Income']-(X['EMI']*1000) 
# Multiply with 1000 to make the units equal 
X_test['Balance Income']=X_test['Total_Income']-(X_test['EMI']*1000)
X=X.drop(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term'], axis=1) 
X_test=X_test.drop(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term'], axis=1)

In [118]:
X.head()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,LoanAmount_log,Total_Income,Total_Income_log,EMI,Balance Income
0,1.0,0,0,1,0,1.0,1,4.852030,5849.0,8.674026,0.355556,5493.444444
1,1.0,1,1,1,0,1.0,0,4.852030,6091.0,8.714568,0.355556,5735.444444
2,1.0,1,0,1,1,1.0,1,4.189655,3000.0,8.006368,0.183333,2816.666667
3,1.0,1,0,0,0,1.0,1,4.787492,4941.0,8.505323,0.333333,4607.666667
4,1.0,0,0,1,0,1.0,1,4.948760,6000.0,8.699515,0.391667,5608.333333


In [119]:
y

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      0
8      1
9      0
10     1
11     1
12     1
13     0
14     1
15     1
16     1
17     0
18     0
19     1
20     0
21     1
22     0
23     0
24     0
25     1
26     1
27     1
28     0
29     1
      ..
584    0
585    0
586    1
587    1
588    1
589    0
590    1
591    0
592    1
593    1
594    1
595    1
596    0
597    0
598    1
599    1
600    0
601    1
602    1
603    1
604    1
605    0
606    1
607    1
608    1
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64

In [122]:
input_num_units = 12
hidden_num_units = 50
output_num_units = 2
epochs = 5
batch_size = 128

In [67]:
from keras.models import Sequential
from keras.layers import Dense

In [123]:
model = Sequential([
  Dense(output_dim=hidden_num_units, input_dim=input_num_units, activation='relu'),
  Dense(output_dim=output_num_units, input_dim=hidden_num_units, activation='softmax'),
])

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=12, activation="relu", units=50)`
  
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, activation="softmax", units=2)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [105]:
from keras.utils import to_categorical
y_binary = to_categorical(y)

In [120]:
y_binary

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [125]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [126]:
trained_model = model.fit(X, y, nb_epoch=epochs, batch_size=batch_size)

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/5
614/614 [==============================] - 0s 289us/step - loss: 884.2073 - accuracy: 0.6873
Epoch 2/5
614/614 [==============================] - 0s 13us/step - loss: 707.7142 - accuracy: 0.6873
Epoch 3/5
614/614 [==============================] - 0s 20us/step - loss: 545.3312 - accuracy: 0.6873
Epoch 4/5
614/614 [==============================] - 0s 20us/step - loss: 382.3408 - accuracy: 0.6873
Epoch 5/5
614/614 [==============================] - 0s 20us/step - loss: 223.0956 - accuracy: 0.6873


In [132]:
pred = model.predict_classes(X_test)

In [131]:
X_test['Dependents']=X_test['Dependents'].replace({0:0,1:1,2:2,'3+':3})

In [137]:
pred=pd.DataFrame(pred)

In [139]:
pred.to_csv(os.path.join(data_dir, 'sub02.csv'), index=False)